In [25]:
import glob
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import math
import numpy as np
import os
import seaborn as sns
from IPython.display import display, Markdown

pd.options.mode.chained_assignment = None

%matplotlib inline
plt.rcParams["figure.figsize"] = [16, 12]

path = "../output/released"
path2 = "../output/validation/tables"

ethnicity_dict = {
    "True": "White",
    "1": "White",
    "2": "Mixed",
    "3": "Asian",
    "4": "Black",
    "5": "Other",
    np.nan: "Unknown",
    "0": "Unknown",
}

imd_dict = {
    "False": "Unknown",
    "0": "Unknown",
    "1": "1 Most deprived",
    "True": "1 Most deprived",
    "2": "2",
    "3": "3",
    "4": "4",
    "5": "5 Least deprived",
}

### Out-of-range Values

The means for CTV3-calculated and SNOMED-calculated BMI derivations by month were often unexpectedly high, with spikes at particular time points. For instance, the mean SNOMED-calculated BMI was 1.8 million in July 2015. To investigate the extent to which the definitions contained unexpected BMI values, values below 4 and above 200 were counted as out-of-range values. A standard BMI chart shows values from 9 to 65, but we have chosen the end points of implausible extremes. The means of out-of-range values were also computed to gauge the magnitude of the errors.

In [26]:
def display_oob(unit,scale):
    df_ct = pd.read_csv(f"{path}/{unit}.csv", index_col=0)
    df_ct.loc[df_ct["category"] == "population", "sort"] = 1
    
    # Rename subcategories 
    df_ct.loc[df_ct["category"] == "ethnicity", "subcategory"] = df_ct["subcategory"].map(ethnicity_dict)
    df_ct.loc[df_ct["category"] == "imd", "subcategory"] = df_ct["subcategory"].map(imd_dict)
    
    # Clean category
    df_ct["category"] = df_ct["category"].apply(lambda x: x.replace("_", " ").title())
    df_ct["subcategory"] = df_ct["subcategory"].apply(lambda x: x.replace("_", " ").title())
    df_ct.loc[df_ct["category"] == "Imd", "category"] = "IMD"
    
    # Sort 
    df_ct = df_ct.sort_values(
        by=["sort","category","subcategory"]
    ).drop(columns=["sort"]).rename(
        columns={
            "category":"Category",
            "subcategory":"Subcategory"
        }
    ).set_index(["Category","Subcategory"])
    
    df_ct = df_ct.replace("-",np.nan)
    
    # Format column values
    for col in df_ct.columns:
        if "count" in col:
            df_ct[col] = df_ct[col].apply(lambda x: np.nan if pd.isnull(x) else round(float(x)))
            df_ct[col] = df_ct[col].apply(lambda x: np.nan if pd.isnull(x) else "{:,.0f}".format(x))
        if scale == "greater":
            if "mean" in col:
                df_ct[col] = df_ct[col].apply(lambda x: np.nan if pd.isnull(x) else "{:,.0f}".format(float(x)))
        if scale == "less":
            if "mean" in col:
                df_ct[col] = df_ct[col].apply(lambda x: np.nan if pd.isnull(x) else round(float(x),2))
                
                
    df_ct = df_ct.fillna("-")

    df_ct = df_ct.rename(
        columns = {
            "count_derived_bmi":"Composite BMI Count",
            "mean_derived_bmi":"Composite BMI Mean",
            "count_recorded_bmi":"SNOMED-recorded BMI Count",
            "mean_recorded_bmi":"SNOMED-recorded BMI Mean",
            "count_computed_bmi":"SNOMED-calculated BMI Count",
            "mean_computed_bmi":"SNOMED-calculated BMI Mean",
            "count_backend_computed_bmi":"CTV3-calculated BMI Count",
            "mean_backend_computed_bmi":"CTV3-calculated BMI Mean",
        }                       
    )
    df_ct.to_csv(f"../output/released/made_locally/local_{unit}.csv")    
    display(df_ct)

In [27]:
display_oob("greater_than_max","greater")

Composite BMI Count  \
Category            Subcategory                                    
Population          All                                  123,265   
Age Band            18-29                                 26,580   
                    30-39                                 23,820   
                    40-49                                 18,790   
                    50-59                                 15,710   
                    60-69                                 14,800   
                    70-79                                 12,805   
                    80+                                   10,760   
                    Missing                                    -   
Dementia            False                                122,475   
                    True                                     790   
Diabetes            False                                110,465   
                    True                                  12,800   
Ethnicity           Asian                                  5,475   
                    Black                                  2,255   
                    Mixed                                  1,240   
                    Other                                  1,835   
                    White                                 73,445   
IMD                 1 Most Deprived                       24,120   
                    2                                     24,220   
                    3                                     25,975   
                    4                                     24,550   
                    5 Least Deprived                      22,425   
                    Unknown                                1,980   
Learning Disability False                                122,715   
                    True                                     550   
Region              East                                  19,390   
                    East Midlands                         14,015   
                    London                                 4,880   
                    North East                             4,130   
                    North West                             8,665   
                    South East                             5,420   
                    South West                            14,420   
                    West Midlands                          3,910   
                    Yorkshire And The Humber               8,865   
Sex                 F                                     73,550   
                    M                                     49,715   

                                             Composite BMI Mean  \
Category            Subcategory                                   
Population          All                              13,811,688   
Age Band            18-29                               376,774   
                    30-39                               189,554   
                    40-49                            89,137,730   
                    50-59                               213,431   
                    60-69                               276,182   
                    70-79                               245,619   
                    80+                                 214,397   
                    Missing                                   -   
Dementia            False                            13,897,094   
                    True                                554,244   
Diabetes            False                            15,380,610   
                    True                                274,133   
Ethnicity           Asian                               278,168   
                    Black                               199,885   
                    Mixed                               185,749   
                    Other                               732,953   
                    White                               272,094   
IMD                 1 Most Deprived                 

In [28]:
display_oob("less_than_min","less")

Composite BMI Count  \
Category            Subcategory                                    
Population          All                                  149,830   
Age Band            18-29                                 34,745   
                    30-39                                 29,180   
                    40-49                                 21,510   
                    50-59                                 19,365   
                    60-69                                 18,125   
                    70-79                                 15,440   
                    80+                                   11,475   
                    Missing                                    -   
Dementia            False                                149,220   
                    True                                     610   
Diabetes            False                                131,520   
                    True                                  18,310   
Ethnicity           Asian                                  6,980   
                    Black                                  2,250   
                    Mixed                                  1,450   
                    Other                                  1,590   
                    White                                 83,160   
IMD                 1 Most Deprived                       29,430   
                    2                                     29,330   
                    3                                     31,210   
                    4                                     29,495   
                    5 Least Deprived                      27,615   
                    Unknown                                2,750   
Learning Disability False                                148,880   
                    True                                     950   
Region              East                                  24,185   
                    East Midlands                         19,980   
                    London                                 5,545   
                    North East                             5,400   
                    North West                             9,280   
                    South East                             8,380   
                    South West                            19,490   
                    West Midlands                          4,250   
                    Yorkshire And The Humber              14,150   
Sex                 F                                     88,370   
                    M                                     61,460   

                                             Composite BMI Mean  \
Category            Subcategory                                   
Population          All                                    1.23   
Age Band            18-29                                  1.56   
                    30-39                                  1.51   
                    40-49                                  1.33   
                    50-59                                   0.5   
                    60-69                                  1.37   
                    70-79                                  1.18   
                    80+                                    0.46   
                    Missing                                   -   
Dementia            False                                  1.23   
                    True                                   1.36   
Diabetes            False                                  1.24   
                    True                                   1.19   
Ethnicity           Asian                                  1.18   
                    Black                                  1.22   
                    Mixed                                  1.48   
                    Other                                  1.21   
                    White                                  1.32   
IMD                 1 Most Deprived                 